## **IMPORTING NECESSARY MODULES**

In [3]:
!pip install tensorflow



[notice] A new release of pip available: 22.2.2 -> 23.1.1
[notice] To update, run: C:\Users\18572\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [4]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
#!pip install rpy2

## **IMPORTING DATA**

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
data = pd.read_csv('Life Expectancy Data.csv')
data.head()

In [ ]:
print('the size of the dataset', data.shape)
print('the columns of the dataset ', list(data.columns))

In [ ]:
data.rename(columns={" BMI ":"BMI","Life expectancy ":"Life_Expectancy","Adult Mortality":"Adult_Mortality",
                   "infant deaths":"Infant_Deaths","percentage expenditure":"Percentage_Exp","Hepatitis B":"HepatitisB",
                  "Measles ":"Measles"," BMI ":"BMI","under-five deaths ":"Under_Five_Deaths","Diphtheria ":"Diphtheria",
                  " HIV/AIDS":"HIV/AIDS"," thinness  1-19 years":"thinness_1to19_years"," thinness 5-9 years":"thinness_5to9_years","Income composition of resources":"Income_Comp_Of_Resources",
                   "Total expenditure":"Tot_Exp"},inplace=True)

In [ ]:
print('Number of Countries', len(data['Country'].unique()))

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.columns

In [ ]:
data.Country.unique()

In [ ]:
#Count of developing and developed countries
data.groupby('Status')["Status"].count()

In [ ]:
data.loc[:, ["Life_Expectancy", "Alcohol"]].groupby(data['Country']).describe().loc[['Morocco','Germany','France','Spain'],:]

In [ ]:
#calculating the null values for each column
data.isna().sum()

## **DATA VISUALIZATION**

In [ ]:
#Distribution of the response variable
data['Life_Expectancy'].hist()

In [ ]:
plt.figure(figsize=(6,6))
plt.bar(data.groupby('Status')['Status'].count().index, data.groupby('Status')['Life_Expectancy'].mean())
plt.xlabel("Status",fontsize=12)
plt.ylabel("Avg Life_Expectancy",fontsize=12)
plt.title("Life_Expectancy w.r.t Status")
plt.show()

In [ ]:
data[data.Country.isin(['Morocco','Germany','Spain','France'])].boxplot(by='Country', 
                       column=['Life_Expectancy'], 
                       grid=False)

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(211)
data[data.Country.isin(['Morocco','Germany','Spain','France'])].groupby('Country')['Life_Expectancy'].mean().sort_values().plot(kind='barh', ax=ax1)

In [ ]:
# Life_Expectancy w.r.t Year using bar plot.
plt.figure(figsize=(7,5))
plt.bar(data.groupby('Year')['Year'].count().index, data.groupby('Year')['Life_Expectancy'].mean(),color='red',alpha=0.65)
plt.xlabel("Year",fontsize=12)
plt.ylabel("Avg Life_Expectancy",fontsize=12)
plt.title("Life_Expectancy w.r.t Year")
plt.show()

In [ ]:
# Using heatmap to observe correlations.
cormat = data.corr()
plt.figure(figsize=(8,8))
sns.heatmap(cormat, square=True, annot=True, linewidths=.5)
plt.title("Correlation matrix among variables")
plt.show()

# **Observation**
* **LifeExpectancy has a negative correlation with adult mortality** 
* **LifeExpectancy has strong correlatios with Schooling and Income composition of resources** 
* **Strong correlation between thinness_1to19_years and thinness_5to9_years** 
* **There is a non-negligible correlation between LifeExpectancy and BMI and body deseases** 

In [ ]:
plt.scatter(data["thinness_5to9_years"], data["thinness_1to19_years"])
plt.title("thinness_1to19_years vs thinness_5to9_years")
plt.show()

In [ ]:
#Strong correlation between "thinness_1to19_years" and "thinness_5to9_years"
data = data.drop(['thinness_5to9_years'], axis=1)

## **DEALING WITH MISSING DATA**

In [ ]:
data.isnull().sum()

In [ ]:
# percentage of null values in each column.
data.isnull().sum()*100/data.isnull().count()

In [ ]:
country_list = data.Country.unique()
fill_list = ['Life_Expectancy','Adult_Mortality','Alcohol','HepatitisB',
             'BMI','Polio','Tot_Exp','Diphtheria','GDP','Population','thinness_1to19_years','Income_Comp_Of_Resources','Schooling']

In [ ]:
# Treat null values using interpolation.
for country in country_list:
    data.loc[data['Country'] == country,fill_list] = data.loc[data['Country'] == country,fill_list].interpolate()

In [ ]:
#Droping rows with null target variable
data[np.isnan(data['Life_Expectancy'])]
data = data.drop(data.index[[624, 769, 1650,1715,1812,1909,1958,2167,2216,2713]])

In [ ]:
def impute_col(row, col): #MCAR
    mean_col = pd.DataFrame({'mean_col':pd.Series(np.round(data.groupby('Country')[col].mean(), 2))})
    if np.isnan(row[col]):
        cnt = row['Country']
        row[col] = mean_col.loc[cnt][0]
    return row

In [ ]:
# Alcohol null values
data[np.isnan(data.Alcohol)]

In [ ]:
data = data.apply(impute_col, args=('Alcohol',) , axis=1)
data = data[data.Country != 'South Sudan']

In [ ]:
# Hepatitis B null values
data[np.isnan(data['HepatitisB'])]

In [ ]:
data = data.drop(['HepatitisB'], axis=1)

In [ ]:
#BMI null values
data[np.isnan(data['BMI'])]

In [ ]:
data = data[data.Country != 'Sudan']

In [ ]:
#Total expenditure null values
data = data.apply(impute_col, args=('Tot_Exp',) , axis=1)

In [ ]:
data = data[~data['Country'].isin(["Democratic People's Republic of Korea","Somalia"])]

In [ ]:
#GDP null values
data = data.apply(impute_col, args=('GDP',) , axis=1)
data

In [ ]:
data = data.drop(['GDP'], axis=1)

In [ ]:
#Population null values
data = data.apply(impute_col, args=('Population',) , axis=1)

In [ ]:
data[np.isnan(data.Population)]['Country'].unique()

In [ ]:
data = data.drop(['Population'], axis=1)

In [ ]:
#Income_Comp_Of_Resources
data = data.apply(impute_col, args=('Income_Comp_Of_Resources',) , axis=1)

In [ ]:
data = data.drop(['Income_Comp_Of_Resources'], axis=1)

In [ ]:
#Schooling
data = data.apply(impute_col, args=('Schooling',) , axis=1)
data = data.drop(['Schooling'], axis=1)

In [ ]:
data.isnull().sum()

# **OUTLIERS**

In [ ]:
# Create a dictionary of columns.
col_dict = {'Life_Expectancy':1, 'Adult_Mortality':2,
            'Infant_Deaths':3, 'Alcohol':4,
            'Percentage_Exp':5,'Measles':6,
            'BMI':7,'Under_Five_Deaths':8,
            'Polio':9,'Tot_Exp':10,
            'Diphtheria':11,'HIV/AIDS':12,
            'thinness_1to19_years':13}

# Detect outliers in each variable using box plots.
plt.figure(figsize=(20,30))

for variable,i in col_dict.items():
                     plt.subplot(5,4,i)
                     plt.boxplot(data[variable])
                     plt.title(variable)

plt.show()

In [ ]:
data.head()

# **FEATURE SELECTION**

In [ ]:
import scipy.stats as stats
stats.ttest_ind(data.loc[data['Status']=='Developed','Life_Expectancy'],data.loc[data['Status']=='Developing','Life_Expectancy'])

**p value is < 0.05
Therefore, the difference of Life Expectancy between Developed and Developing countries is significant. We can consider 'Status' as a feature.**

**Also "Adult mortality" : -0.7, "HIV/AIDS" : -0.56,  "BMI" : 0.57, "Polio" : 0.47, "GDP" : 0.46, "Alcohol" : 0.4, "thinness_1to19_years" : 0.45**

# **FEATURE ENGINEERING**

* **DUMMIFICATION OF STATUS**
* **NORMALIZING NUMERICAL FEATURES**

In [ ]:
feature_df = data[['Country','Status','Adult_Mortality','Alcohol','HIV/AIDS','Polio','BMI', 'thinness_1to19_years','Life_Expectancy']]

In [ ]:
feature_df = pd.concat([feature_df,pd.get_dummies(feature_df['Status'],drop_first=True)],axis=1)
final = feature_df.drop('Status',axis=1)

In [ ]:
final['Adult_Mortality_scaled'] =final['Adult_Mortality'].apply(lambda x : ((x-np.min(final['Adult_Mortality']))/(np.max(final['Adult_Mortality'])-np.min(final['Adult_Mortality'])) * (20)))

In [ ]:
final['Polio_scaled'] =final['Polio'].apply(lambda x : ((x-np.min(final['Polio']))/(np.max(final['Polio'])-np.min(final['Polio'])) * (20)))

In [ ]:
final.to_csv('./final.csv',index = False) 

In [ ]:
final = pd.read_csv('./final.csv')

In [ ]:
final.head()

# **EMBED THE COUNTRY FEATURE**

In [ ]:
countries = final.Country.unique()
country_dict = {'countries': list(countries)}
country_df = pd.DataFrame(country_dict)

In [ ]:
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    return feature_layer(country_dict).numpy()

In [ ]:
countries = feature_column.categorical_column_with_vocabulary_list(
 'countries', country_df['countries'])

In [ ]:
countries_embedding = feature_column.embedding_column(countries, dimension=4)

In [ ]:
countries_embedding =  demo(countries_embedding)

In [ ]:
b = []
for embed in countries_embedding:
    b.extend([embed] * 16)

In [ ]:
final['countries_embedding'] = pd.Series(b)

In [ ]:
final['sum_countries_embedding'] = final['countries_embedding'].apply(lambda x: x.sum())

In [ ]:
final =  final.rename(columns={"HIV/AIDS": "hivaids"})

In [ ]:
feature_df = ['sum_countries_embedding','Adult_Mortality_scaled','Alcohol','hivaids','Polio_scaled','BMI', 'thinness_1to19_years','Developing']

In [ ]:
final

# **TRAINING**

In [ ]:
# split the data into training and testing sets
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# df = pd.read_csv('Life Expectancy Data.csv')
X = final.loc[:,feature_df]
y = final.Life_Expectancy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train and evaluate linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_mse = mean_squared_error(y_test, lr_pred)
lr_r2 = lr.score(X_test, y_test)

# train and evaluate mixed effects model
# (assuming the data has a nested structure with country as a random effect)
mixed_model = smf.mixedlm("Life_Expectancy ~ Adult_Mortality_scaled + Alcohol + Polio_scaled + hivaids + BMI + thinness_1to19_years + Developing", 
                 final, groups=final["sum_countries_embedding"], re_formula="~Adult_Mortality_scaled + Alcohol + Polio_scaled + hivaids + BMI + thinness_1to19_years + Developing") 
mixed_results = mixed_model.fit()
mixed_pred = mixed_results.predict(X_test)
mixed_mse = mean_squared_error(y_test, mixed_pred)
mixed_r2 = sm.OLS(y_test, mixed_pred).fit().rsquared

# train and evaluate neural network model
nn = MLPRegressor(hidden_layer_sizes=(100,), activation="relu", solver="adam", max_iter=1000)
nn.fit(X_train, y_train)
nn_pred = nn.predict(X_test)
nn_mse = mean_squared_error(y_test, nn_pred)
nn_r2 = nn.score(X_test, y_test)

# create a Random Forest Regressor object and set the hyperparameters
rf_reg = RandomForestRegressor(n_estimators = 100, max_depth = 10, random_state = 42)
rf_reg.fit(X_train, y_train)
rf_reg_pred = rf_reg.predict(X_test)
rf_reg_mse = mean_squared_error(y_test, rf_reg_pred)
rf_reg_r2 = rf_reg.score(X_test, y_test)

# print the MSE for each model
print("Linear Regression MSE:", lr_mse)
print("Mixed Effects Model MSE:", mixed_mse)
print("Neural Network MSE:", nn_mse)
print("Random Forest Regressor MSE:", rf_reg_mse)

# print the R_Squared value for each model
print("Linear Regression R_2:", lr_r2)
print("Mixed Effects Model R_2:", mixed_r2)
print("Neural Network R_2:", nn_r2)
print("Random Forest Regressor R_2:", rf_reg_r2)

In [ ]:
df_scores = pd.DataFrame([['Linear Regression', lr_mse, lr_r2], ['Mixed Effects Model', mixed_mse, mixed_r2], ['Neural Network', nn_mse, nn_r2],
                                                            ['Random Forest Regressor', rf_reg_mse, rf_reg_r2]], columns = ['Model Name', 'MSE_Value', 'R2_Value'])
df_scores

The table provides the results of four different models, where each model has been evaluated based on two metrics: mean squared error (MSE) and R-squared value.

MSE measures the average squared difference between the predicted values and the actual values of the dependent variable. A lower MSE indicates better performance of the model in predicting the dependent variable.

R-squared measures the proportion of variance in the dependent variable that is explained by the independent variables in the model. A higher R-squared value indicates better fit of the model to the data.

Looking at the table, we can see that the Random Forest Regressor model has the lowest MSE value (4.153755) compared to other models, indicating that this model has the best performance in predicting the dependent variable. The Mixed Effects Model has the highest MSE value (75.198735), which indicates relatively poor performance in predicting the dependent variable.

Moreover, the table shows that the Mixed Effects Model has the highest R-squared value (0.984771), which indicates that this model explains a large proportion of the variance in the dependent variable. The Linear Regression and Neural Network models also have high R-squared values (0.700833 and 0.880466, respectively), but they are lower than the R-squared value of the Mixed Effects Model. The Random Forest Regressor has an R-squared value of 0.950499, which is lower than the Mixed Effects Model but still indicates good performance in explaining the variance in the dependent variable.

Overall, it is important to evaluate both MSE and R-squared values when comparing models, as they provide complementary information on the performance and fit of the models to the data.

In [ ]:
# create a list of predicted values and actual values for each model
lr_cumulative = np.cumsum(np.sort(lr_pred)[::-1])
lr_percentage = 100 * lr_cumulative / np.sum(y_test)
mixed_cumulative = np.cumsum(np.sort(mixed_pred)[::-1])
mixed_percentage = 100 * mixed_cumulative / np.sum(y_test)
nn_cumulative = np.cumsum(np.sort(nn_pred)[::-1])
nn_percentage = 100 * nn_cumulative / np.sum(y_test)
rf_cumulative = np.cumsum(np.sort(rf_reg_pred)[::-1])
rf_percentage = 100 * rf_cumulative / np.sum(y_test)

# plot the gains chart for each model
plt.plot(np.arange(0, 101, 10), lr_percentage[::57], label='Linear Regression')
plt.plot(np.arange(0, 101, 10), mixed_percentage[::57], label='Mixed Effects Model')
plt.plot(np.arange(0, 101, 10), nn_percentage[::57], label='Neural Network')
plt.plot(np.arange(0, 101, 10), rf_percentage[::57], label='Random Forest Regressor')
plt.plot([0, 100], [0, 100], linestyle='--', label='Baseline')

plt.title('Gains Chart')
plt.legend()
plt.tight_layout()
plt.show()

## **LINEAR REGRESSION**

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)
x = final.loc[:,feature_df]
y = final.Life_Expectancy
model.fit(x, y)

In [ ]:
print("Model slopes:    ", model.coef_)
print("Model intercept:", model.intercept_)

In [ ]:
y_predict = model.predict(x.values)
RMSE = np.sqrt(((y-y_predict)**2).values.mean())

results = pd.DataFrame()
results["Method"] = ["Linear Regression"]
results["RMSE"] = RMSE
results

In [ ]:
np.min(final['Life_Expectancy']), np.max(final['Life_Expectancy'])

In [ ]:
!pip install yellowbrick


In [ ]:
from yellowbrick.regressor import ResidualsPlot
visualizer = ResidualsPlot(estimator = model)
visualizer.fit(x, y) 
visualizer.poof()

**Residuals should be pretty symmetrically distributed, tending to cluster towards the middle of the plot. Clustered around the lower single digits of the y-axis.
In general, no clear patterns.**

## **MIXED EFFECT MODEL**

In [ ]:
!pip install statsmodels

In [ ]:
#!pip install -q statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
md = smf.mixedlm("Life_Expectancy ~ Adult_Mortality_scaled + Alcohol + Polio_scaled + hivaids + BMI + thinness_1to19_years + Developing", 
                 final, 
                 groups=final["sum_countries_embedding"], re_formula="~Adult_Mortality_scaled + Alcohol + Polio_scaled + hivaids + BMI + thinness_1to19_years + Developing") 
#re_formula To ensure that each country has its own random slope

In [ ]:
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

In [ ]:
performance = pd.DataFrame()
performance["residuals"] = mdf.resid.values
performance["predicted"] = mdf.fittedvalues

sns.lmplot(x = "predicted", y = "residuals", data = performance)

In [ ]:
y_predict = mdf.fittedvalues
RMSE = np.sqrt(((y-y_predict)**2).values.mean())
results.loc[3] = ["Mixed_Random_Slopes", RMSE]
results

# **USING A NEURAL NETWORK**

In [ ]:
final = pd.read_csv('./final.csv')

In [ ]:
final.head()

In [ ]:
final['Status'] = final['Developing'].map(lambda x: 'Developing' if x==1 else 'Developed')

In [ ]:
train, test = train_test_split(final, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

In [ ]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('Life_Expectancy')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [ ]:
train_ds = df_to_dataset(train)
val_ds = df_to_dataset(val, shuffle=False, batch_size=16)
test_ds = df_to_dataset(test, shuffle=False, batch_size=16)

In [ ]:
final_batch = next(iter(train_ds))[0]

In [ ]:
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    return feature_layer(final_batch).numpy()

In [ ]:
feature_columns = []
# numeric cols
for header in ['Alcohol', 'HIV/AIDS', 'Polio_scaled','BMI','thinness_1to19_years']:
    feature_columns.append(feature_column.numeric_column(header))

In [ ]:
#catgorical cols
status = feature_column.categorical_column_with_vocabulary_list(
      'Status', ['Developing', 'Developed'])

status = feature_column.indicator_column(status)
feature_columns.append(status)

In [ ]:
# embedding columns
country = feature_column.categorical_column_with_vocabulary_list(
      'Country', final.Country.unique())
country_embedding = feature_column.embedding_column(country, dimension=6)
feature_columns.append(country_embedding)

In [ ]:
#Input layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
#Model architecture
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(32, activation='relu'), 
  layers.Dense(16, activation='relu'), 
  layers.Dense(1)
])
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(0.001), metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
#Fitting
history = model.fit(train_ds, validation_data=val_ds, epochs=40)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
predictions = model.predict(test_ds)
y = np.concatenate([y for x, y in test_ds], axis=0)

In [ ]:
compare = pd.DataFrame({'predictions':predictions.reshape((-1,)), 'True': y})
compare.tail()

In [ ]:
model_err = model.evaluate(test_ds)